In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from phik import phik_matrix
import category_encoders as ce
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
#from statsmodels.tsa.arima.model import ARIMA
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from datetime import timedelta


import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re


pd.set_option('display.max_columns', 30)
warnings.filterwarnings("ignore")

In [2]:
from catboost import CatBoostRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
from prophet import Prophet

In [4]:
sales_df_train = pd.read_csv('D:\\yandex\\sp_sales_task\\sales_df_train.csv')
pr_df = pd.read_csv('D:\\yandex\\sp_sales_task\\pr_df.csv')
st_df = pd.read_csv('D:\\yandex\\sp_sales_task\\st_df.csv')


In [5]:
sales_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883015 entries, 0 to 883014
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   date                     883015 non-null  object 
 3   pr_sales_type_id         883015 non-null  int64  
 4   pr_sales_in_units        883015 non-null  float64
 5   pr_promo_sales_in_units  883015 non-null  float64
 6   pr_sales_in_rub          883015 non-null  float64
 7   pr_promo_sales_in_rub    883015 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 53.9+ MB


In [6]:
sales_submission = pd.read_csv('D:\\yandex\\sp_sales_task\\sales_submission.csv')

In [7]:
st_df.info()                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   st_id              12 non-null     object
 1   st_city_id         12 non-null     object
 2   st_division_code   12 non-null     object
 3   st_type_format_id  12 non-null     int64 
 4   st_type_loc_id     12 non-null     int64 
 5   st_type_size_id    12 non-null     int64 
 6   st_is_active       12 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 804.0+ bytes


In [8]:
full_df = sales_df_train.merge(st_df, on='st_id')


In [9]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883015 entries, 0 to 883014
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   date                     883015 non-null  object 
 3   pr_sales_type_id         883015 non-null  int64  
 4   pr_sales_in_units        883015 non-null  float64
 5   pr_promo_sales_in_units  883015 non-null  float64
 6   pr_sales_in_rub          883015 non-null  float64
 7   pr_promo_sales_in_rub    883015 non-null  float64
 8   st_city_id               883015 non-null  object 
 9   st_division_code         883015 non-null  object 
 10  st_type_format_id        883015 non-null  int64  
 11  st_type_loc_id           883015 non-null  int64  
 12  st_type_size_id          883015 non-null  int64  
 13  st_is_active             883015 non-null  int64  
dtypes: f

In [10]:
pr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pr_sku_id     2050 non-null   object
 1   pr_group_id   2050 non-null   object
 2   pr_cat_id     2050 non-null   object
 3   pr_subcat_id  2050 non-null   object
 4   pr_uom_id     2050 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 80.2+ KB


In [11]:
full_df = full_df.merge(pr_df, on='pr_sku_id')

In [12]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 883015 entries, 0 to 883014
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   date                     883015 non-null  object 
 3   pr_sales_type_id         883015 non-null  int64  
 4   pr_sales_in_units        883015 non-null  float64
 5   pr_promo_sales_in_units  883015 non-null  float64
 6   pr_sales_in_rub          883015 non-null  float64
 7   pr_promo_sales_in_rub    883015 non-null  float64
 8   st_city_id               883015 non-null  object 
 9   st_division_code         883015 non-null  object 
 10  st_type_format_id        883015 non-null  int64  
 11  st_type_loc_id           883015 non-null  int64  
 12  st_type_size_id          883015 non-null  int64  
 13  st_is_active             883015 non-null  int64  
 14  pr_g

In [13]:
full_df.head(20)

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id
0,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-10-20,1,5.0,5.0,825.0,825.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
1,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2023-02-02,1,1.0,1.0,107.0,107.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
2,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-10-09,1,1.0,1.0,167.0,167.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
3,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2023-06-22,0,0.0,0.0,79.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
4,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2023-06-30,1,25.0,25.0,3766.0,3766.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
5,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-12-29,0,49.0,0.0,9143.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
6,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-08-02,1,35.0,35.0,4829.0,4829.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
7,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-09-17,0,1.0,0.0,117.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
8,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2023-01-05,0,13.0,0.0,2332.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
9,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-11-14,0,12.0,0.0,2159.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1


In [14]:
full_df['date'] = pd.to_datetime(full_df['date'])

In [15]:
full_df['day_of_week'] = full_df['date'].dt.dayofweek
full_df['month'] = full_df['date'].dt.month

In [16]:
full_df.sort_values(by=['st_id', 'pr_sku_id', 'date'], inplace=True)

In [17]:
full_df[full_df['pr_uom_id'] == 1]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month
548747,084a8a9aa8cced9175bd07bc44998e75,c4a665596d4f67cecb7542c9fad407ee,2023-05-01,0,2.0,0.0,287.0,0.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c20ad4d76fe97759aa27a0c99bff6710,fb2fcd534b0ff3bbed73cc51df620323,45551f4c29881972ae5ed3666a5b1b8b,1,0,5
548746,084a8a9aa8cced9175bd07bc44998e75,c4a665596d4f67cecb7542c9fad407ee,2023-05-06,1,3.0,3.0,405.0,405.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c20ad4d76fe97759aa27a0c99bff6710,fb2fcd534b0ff3bbed73cc51df620323,45551f4c29881972ae5ed3666a5b1b8b,1,5,5
738982,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,2023-07-13,0,1.0,0.0,81.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,130d817d8b3f616e97c555ec57946aec,1,3,7
738983,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,2023-07-14,0,2.0,0.0,153.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,130d817d8b3f616e97c555ec57946aec,1,4,7
738984,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,2023-07-15,0,2.0,0.0,155.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,130d817d8b3f616e97c555ec57946aec,1,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855609,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-07-14,0,6.0,0.0,234.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,4,7
855608,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-07-15,0,1.0,0.0,21.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,5,7
855618,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-07-16,0,3.0,0.0,115.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,6,7
855631,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-07-17,0,2.0,0.0,82.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,0,7


In [18]:
pr_df[pr_df['pr_uom_id']==1]

,pr_sku_id,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id
36,ae0ea26efd73c891f6fe4f8af687c88f,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1
37,ebef63de0e8fe04e0d681a25face55c9,6512bd43d9caa6e02c990b0a82652dca,b59c67bf196a4758191e42f76670ceba,a09fe216278f103e70a7a179e173831c,1
38,e983f56659b43d6691446b013c0b9ef1,aab3238922bcc25a6f606eb525ffdc56,6d9c547cf146054a5a720606a7694467,8b51a5273e5c6883ba614922c5c9e867,1
39,c4a665596d4f67cecb7542c9fad407ee,c20ad4d76fe97759aa27a0c99bff6710,fb2fcd534b0ff3bbed73cc51df620323,45551f4c29881972ae5ed3666a5b1b8b,1
41,90702dbda20da8380fb559b1ea8c0140,c51ce410c124a10e0db5e4b97fc2af39,535ab76633d94208236a2e829ea6d888,5fe86eaa61bb6af68ee6c47fccf3e6a5,1
...,...,...,...,...,...
2039,d962f4c1389fcacf76200b0fd64a3308,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,75b64073c609a818b642755b8884313f,1
2044,40db0970677f1f9ba660753f0baa7d00,aab3238922bcc25a6f606eb525ffdc56,4edaa105d5f53590338791951e38c3ad,2835e8a68be5daab66d9385f24b549d7,1
2045,2c3b895d9fccc33226fee2c521618893,c74d97b01eae257e44aa9d5bade97baf,a2cc63e065705fe938a4dda49092966f,4589b8f516d28558b3850c37776cadc8,1
2046,6058d6b7d1f04803d15d1896d3256c92,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c0a667df8f0d6ee98dda972b0870fabf,1


In [19]:
full_df.describe()

,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_uom_id,day_of_week,month
count,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000,883015.000000
mean,0.406486,5.037956,2.230010,596.705181,294.766561,1.096438,1.206916,11.464332,0.999174,6.751821,3.095275,6.564153
std,0.491178,13.402984,11.063288,1577.366179,1402.764168,0.415505,0.438867,4.509462,0.028721,7.677614,1.977901,3.490489
min,0.000000,-39.000000,-39.000000,-3373.000000,-3373.000000,1.000000,1.000000,8.000000,0.000000,1.000000,0.000000,1.000000
25%,0.000000,1.000000,0.000000,98.000000,0.000000,1.000000,1.000000,8.000000,1.000000,1.000000,1.000000,4.000000
50%,0.000000,2.000000,0.000000,224.000000,0.000000,1.000000,1.000000,12.000000,1.000000,1.000000,3.000000,6.000000
75%,1.000000,4.000000,1.000000,540.000000,127.000000,1.000000,1.000000,12.000000,1.000000,17.000000,5.000000,10.000000
max,1.000000,2348.000000,2348.000000,183265.000000,183265.000000,4.000000,3.000000,32.000000,1.000000,17.000000,6.000000,12.000000


In [20]:
full_df[full_df['pr_sales_in_rub'] < 0].count()

st_id                      78
pr_sku_id                  78
date                       78
pr_sales_type_id           78
pr_sales_in_units          78
pr_promo_sales_in_units    78
pr_sales_in_rub            78
pr_promo_sales_in_rub      78
st_city_id                 78
st_division_code           78
st_type_format_id          78
st_type_loc_id             78
st_type_size_id            78
st_is_active               78
pr_group_id                78
pr_cat_id                  78
pr_subcat_id               78
pr_uom_id                  78
day_of_week                78
month                      78
dtype: int64

In [21]:
full_df = full_df[full_df['pr_sales_in_rub'] >= 0]

In [22]:
full_df[full_df['pr_sales_in_rub'] < 0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month


In [23]:
full_df[full_df['pr_sales_in_rub'] >100000]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month
69906,16a5cdae362b8d27a1d8f8c7b78b4330,4e824a65fbd01f1708131bd247842688,2022-12-30,1,290.0,290.0,156241.0,156241.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,f633b7df336a0a7f8251e1eec56ae973,1,4,12
553399,42a0e188f5033bc65bf8d78622277c4e,54cc81ad9eadc5ccde13e001ddfa3eb1,2022-12-24,1,532.0,532.0,112555.0,112555.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f9ab16852d455ce9203da64f4fc7f92d,f88e7dda1d4050831666d96461fe674c,1,5,12
553560,42a0e188f5033bc65bf8d78622277c4e,54cc81ad9eadc5ccde13e001ddfa3eb1,2022-12-29,1,417.0,417.0,113163.0,113163.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f9ab16852d455ce9203da64f4fc7f92d,f88e7dda1d4050831666d96461fe674c,1,3,12
779436,42a0e188f5033bc65bf8d78622277c4e,8cad09283d1d6c5fb08daec8a576a72d,2022-08-02,1,239.0,239.0,111397.0,111397.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,1,8
779401,42a0e188f5033bc65bf8d78622277c4e,8cad09283d1d6c5fb08daec8a576a72d,2022-08-08,0,345.0,0.0,169379.0,0.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,0,8
779404,42a0e188f5033bc65bf8d78622277c4e,8cad09283d1d6c5fb08daec8a576a72d,2023-03-21,1,440.0,440.0,183265.0,183265.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,1,3
779527,42a0e188f5033bc65bf8d78622277c4e,8cad09283d1d6c5fb08daec8a576a72d,2023-04-15,1,270.0,270.0,112300.0,112300.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,5,4
656569,42a0e188f5033bc65bf8d78622277c4e,a70a600d7c593ca50cc6639cd64ed71a,2022-12-30,1,185.0,185.0,107865.0,107865.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,2df45244f09369e16ea3f9117ca45157,a6b626d2738e74999cc3fdbdddfdb49f,1,4,12
707509,42a0e188f5033bc65bf8d78622277c4e,e5d867b2770e0aade8653a4893ec6160,2023-04-17,1,530.0,530.0,105971.0,105971.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,60787c41b04097dfea76addfccd12243,17,0,4


In [24]:
full_df[full_df['pr_sales_in_units'] == 0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month
168158,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-14,1,0.0,0.0,67.0,67.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8
168110,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-28,1,0.0,0.0,104.0,104.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8
168171,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-28,0,0.0,0.0,102.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8
168363,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-09-04,1,0.0,0.0,109.0,109.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,9
168345,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-09-14,1,0.0,0.0,61.0,61.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855629,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-24,0,0.0,0.0,13.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,2,8
855599,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-28,1,0.0,0.0,14.0,14.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,6,8
855589,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-30,0,0.0,0.0,14.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,1,8
855596,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-06-16,1,0.0,0.0,21.0,21.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,4,6


In [25]:
full_df = full_df[full_df['pr_sales_in_rub'] < 10000]

In [26]:
# создаем колонку с ценой за одну единицу

full_df['one_unit_price'] = full_df.apply(
    lambda row: row['pr_sales_in_rub'] / row['pr_sales_in_units'] 
                if (row['pr_sales_in_rub'] != 0 and 
                    row['pr_sales_in_units'] != 0 and 
                    row['pr_promo_sales_in_rub'] == 0) 
                else None, axis=1)



In [27]:
full_df[full_df['pr_promo_sales_in_rub'] !=0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month,one_unit_price
882986,084a8a9aa8cced9175bd07bc44998e75,0376a60d9a7ce7965beddc4815588697,2023-04-07,1,1.0,1.0,81.0,81.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,17,4,4,NaN
882987,084a8a9aa8cced9175bd07bc44998e75,0376a60d9a7ce7965beddc4815588697,2023-04-11,1,1.0,1.0,81.0,81.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,17,1,4,NaN
882982,084a8a9aa8cced9175bd07bc44998e75,0376a60d9a7ce7965beddc4815588697,2023-04-12,1,1.0,1.0,81.0,81.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,17,2,4,NaN
882985,084a8a9aa8cced9175bd07bc44998e75,0376a60d9a7ce7965beddc4815588697,2023-06-24,1,1.0,1.0,132.0,132.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,17,5,6,NaN
883014,084a8a9aa8cced9175bd07bc44998e75,61ede1b712ff37c91101eed14b7a6c4f,2023-04-11,1,1.0,1.0,120.0,120.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,4,3,19,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,17,1,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652246,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff62e7bffaca5e0ac8029cd29298c8c3,2022-12-20,1,2.0,2.0,49.0,49.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17,1,12,NaN
652214,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff62e7bffaca5e0ac8029cd29298c8c3,2022-12-23,1,2.0,2.0,54.0,54.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17,4,12,NaN
652251,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff62e7bffaca5e0ac8029cd29298c8c3,2022-12-25,1,2.0,2.0,51.0,51.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17,6,12,NaN
652284,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff62e7bffaca5e0ac8029cd29298c8c3,2023-01-04,1,1.0,1.0,27.0,27.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17,2,1,NaN


In [28]:
# добавляем цену за одну единицу по промо 

full_df['one_unit_price'] = full_df.apply(
    lambda row: row['pr_promo_sales_in_rub'] / row['pr_promo_sales_in_units']
                 if (np.isnan(row['one_unit_price']) and
                     row['pr_promo_sales_in_rub'] != 0 and
                     row['pr_promo_sales_in_units'] != 0)
                 else row['one_unit_price'],
    axis=1
)


In [29]:
full_df[full_df['pr_sales_in_units'] == 0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month,one_unit_price
168158,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-14,1,0.0,0.0,67.0,67.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8,NaN
168110,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-28,1,0.0,0.0,104.0,104.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8,NaN
168171,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-08-28,0,0.0,0.0,102.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,8,NaN
168363,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-09-04,1,0.0,0.0,109.0,109.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,6,9,NaN
168345,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2022-09-14,1,0.0,0.0,61.0,61.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,8afe22eeb3f3f68de994a3c60388858c,1,2,9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855629,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-24,0,0.0,0.0,13.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,2,8,NaN
855599,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-28,1,0.0,0.0,14.0,14.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,6,8,NaN
855589,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2022-08-30,0,0.0,0.0,14.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,1,8,NaN
855596,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ff5cb535afe362f766ca6bd5d8e92e88,2023-06-16,1,0.0,0.0,21.0,21.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1,4,6,NaN


In [30]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879062 entries, 882986 to 652221
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   st_id                    879062 non-null  object        
 1   pr_sku_id                879062 non-null  object        
 2   date                     879062 non-null  datetime64[ns]
 3   pr_sales_type_id         879062 non-null  int64         
 4   pr_sales_in_units        879062 non-null  float64       
 5   pr_promo_sales_in_units  879062 non-null  float64       
 6   pr_sales_in_rub          879062 non-null  float64       
 7   pr_promo_sales_in_rub    879062 non-null  float64       
 8   st_city_id               879062 non-null  object        
 9   st_division_code         879062 non-null  object        
 10  st_type_format_id        879062 non-null  int64         
 11  st_type_loc_id           879062 non-null  int64         
 12  st_type_siz

In [31]:
full_df[(full_df['pr_sales_in_rub'] == 0) & (full_df['pr_sales_in_units'] == 0)]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month,one_unit_price
383040,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,2022-11-01,0,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,1,11,NaN
634916,16a5cdae362b8d27a1d8f8c7b78b4330,04bbb07b1057b09d04209991f3eadd8f,2023-02-16,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,46771d1f432b42343f56f791422a4991,38cc804f2e8d4b18785b84c68517eadf,1,3,2,NaN
704960,16a5cdae362b8d27a1d8f8c7b78b4330,06076bf3c802444ebc7d5619ab64ceb6,2022-11-11,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,4,11,NaN
705051,16a5cdae362b8d27a1d8f8c7b78b4330,06076bf3c802444ebc7d5619ab64ceb6,2023-07-04,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,1,7,NaN
488838,16a5cdae362b8d27a1d8f8c7b78b4330,0a68dd43c227b0e66d52665d6c3ca8ba,2022-10-29,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,5,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627026,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fbc15b8efc0a6d6eaae607c074816644,2022-11-13,0,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f9ab16852d455ce9203da64f4fc7f92d,f24d451b1a893d2b679152e43db2b29d,1,6,11,NaN
125650,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fbf6c91454d7c3cea7b03f3092cbfb73,2023-01-21,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,567115097ac405a46a84ef641c2d7069,1,5,1,NaN
402008,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fd519eb13005448a303df1089505a42b,2022-09-16,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f862d13454fd267baa5fedfffb200567,76edb781a6decc9f7dd7e94622f857e6,1,4,9,NaN
187281,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fdb89dfd1a27256904eb5a5734252102,2023-02-15,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,567115097ac405a46a84ef641c2d7069,1,2,2,NaN


In [32]:
full_df['one_unit_price'] = full_df.groupby('pr_sku_id')['one_unit_price'].transform(lambda x: x.ffill())

In [33]:
# заполняем нули в pr_sales_in_units 

full_df['pr_sales_in_units'] = full_df.apply(
    lambda row: row['pr_sales_in_rub'] / row['one_unit_price'] 
                if row['pr_sales_in_units'] == 0 
                else row['pr_sales_in_units'], axis=1)

In [34]:
# заполняем нули в pr_sales_in_rub

full_df['pr_sales_in_rub'] = full_df.apply(
    lambda row: row['pr_sales_in_units'] * row['one_unit_price'] 
                if row['pr_sales_in_rub'] == 0 
                else row['pr_sales_in_rub'], axis=1)

In [35]:
new_df = full_df[(full_df['pr_sales_in_rub'] != 0) & (full_df['pr_sales_in_units'] != 0)]

In [36]:
full_df[full_df['pr_sales_in_rub'] == 0]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month,one_unit_price
383040,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,2022-11-01,0,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,1,11,151.000000
634916,16a5cdae362b8d27a1d8f8c7b78b4330,04bbb07b1057b09d04209991f3eadd8f,2023-02-16,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,46771d1f432b42343f56f791422a4991,38cc804f2e8d4b18785b84c68517eadf,1,3,2,142.000000
704960,16a5cdae362b8d27a1d8f8c7b78b4330,06076bf3c802444ebc7d5619ab64ceb6,2022-11-11,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,4,11,414.800000
705051,16a5cdae362b8d27a1d8f8c7b78b4330,06076bf3c802444ebc7d5619ab64ceb6,2023-07-04,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c51ce410c124a10e0db5e4b97fc2af39,5caf41d62364d5b41a893adc1a9dd5d4,4e96157c3c8c24f4761a7bc4411a2504,1,1,7,334.142857
488838,16a5cdae362b8d27a1d8f8c7b78b4330,0a68dd43c227b0e66d52665d6c3ca8ba,2022-10-29,1,0.0,0.0,0.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,6512bd43d9caa6e02c990b0a82652dca,e58cc5ca94270acaceed13bc82dfedf7,c93c26cd49ea08e6b8984351d4164575,1,5,10,143.314286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627026,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fbc15b8efc0a6d6eaae607c074816644,2022-11-13,0,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f9ab16852d455ce9203da64f4fc7f92d,f24d451b1a893d2b679152e43db2b29d,1,6,11,337.000000
125650,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fbf6c91454d7c3cea7b03f3092cbfb73,2023-01-21,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,567115097ac405a46a84ef641c2d7069,1,5,1,236.000000
402008,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fd519eb13005448a303df1089505a42b,2022-09-16,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,32bb90e8976aab5298d5da10fe66f21d,f862d13454fd267baa5fedfffb200567,76edb781a6decc9f7dd7e94622f857e6,1,4,9,94.250000
187281,fa7cdfad1a5aaf8370ebeda47a1ff1c3,fdb89dfd1a27256904eb5a5734252102,2023-02-15,1,0.0,0.0,0.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,567115097ac405a46a84ef641c2d7069,1,2,2,157.000000


In [37]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878614 entries, 882986 to 652221
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   st_id                    878614 non-null  object        
 1   pr_sku_id                878614 non-null  object        
 2   date                     878614 non-null  datetime64[ns]
 3   pr_sales_type_id         878614 non-null  int64         
 4   pr_sales_in_units        877962 non-null  float64       
 5   pr_promo_sales_in_units  878614 non-null  float64       
 6   pr_sales_in_rub          878605 non-null  float64       
 7   pr_promo_sales_in_rub    878614 non-null  float64       
 8   st_city_id               878614 non-null  object        
 9   st_division_code         878614 non-null  object        
 10  st_type_format_id        878614 non-null  int64         
 11  st_type_loc_id           878614 non-null  int64         
 12  st_type_siz

In [38]:
new_df['date'].min()

Timestamp('2022-08-01 00:00:00')

In [39]:
new_df['date'].max()

Timestamp('2023-07-18 00:00:00')

In [40]:
data = pd.read_csv('D:\Загрузки хром\prazdniki.csv')

In [41]:
data

,date,is_weekend
0,01.08.2022,Рабочий день
1,02.08.2022,Рабочий день
2,03.08.2022,Рабочий день
3,04.08.2022,Рабочий день
4,05.08.2022,Рабочий день
...,...,...
347,14.07.2023,Рабочий день
348,15.07.2023,Выходной день
349,16.07.2023,Выходной день
350,17.07.2023,Рабочий день


In [42]:
data.loc[data['is_weekend'] == 'Рабочий день', 'is_weekend'] = 0

In [43]:
data.loc[data['is_weekend'] != 'Рабочий день', 'is_weekend'] = 1 

In [44]:
data['date'] = pd.to_datetime(data['date'])

In [45]:
new_df = new_df.merge(data, on='date', how ='left')

In [46]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 878614 entries, 0 to 878613
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   st_id                    878614 non-null  object        
 1   pr_sku_id                878614 non-null  object        
 2   date                     878614 non-null  datetime64[ns]
 3   pr_sales_type_id         878614 non-null  int64         
 4   pr_sales_in_units        877962 non-null  float64       
 5   pr_promo_sales_in_units  878614 non-null  float64       
 6   pr_sales_in_rub          878605 non-null  float64       
 7   pr_promo_sales_in_rub    878614 non-null  float64       
 8   st_city_id               878614 non-null  object        
 9   st_division_code         878614 non-null  object        
 10  st_type_format_id        878614 non-null  int64         
 11  st_type_loc_id           878614 non-null  int64         
 12  st_type_size_id 

In [47]:
new_df[new_df['pr_sales_in_units'].isna()]

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id,day_of_week,month,one_unit_price,is_weekend
1239,16a5cdae362b8d27a1d8f8c7b78b4330,02713435f3587e2c81d8f6a9016763ea,2022-10-06,0,NaN,0.0,144.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,1,3,10,NaN,NaN
1240,16a5cdae362b8d27a1d8f8c7b78b4330,02713435f3587e2c81d8f6a9016763ea,2022-10-07,1,NaN,0.0,124.0,124.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,1,4,10,NaN,NaN
1241,16a5cdae362b8d27a1d8f8c7b78b4330,02713435f3587e2c81d8f6a9016763ea,2022-10-09,0,NaN,0.0,116.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,1,6,10,NaN,1
1242,16a5cdae362b8d27a1d8f8c7b78b4330,02713435f3587e2c81d8f6a9016763ea,2022-10-11,0,NaN,0.0,134.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,8feef08314d477a5b68ba18a6d35819d,1,1,10,NaN,1
1603,16a5cdae362b8d27a1d8f8c7b78b4330,040a02c2ad1561cbcfc9cae5b4c4b73b,2022-08-22,0,NaN,0.0,78.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,c01c475ac8d447730ece97d846cc4f25,1,0,8,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784232,fa7cdfad1a5aaf8370ebeda47a1ff1c3,2c3b895d9fccc33226fee2c521618893,2022-08-25,0,NaN,0.0,173.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,a2cc63e065705fe938a4dda49092966f,4589b8f516d28558b3850c37776cadc8,1,3,8,NaN,1
790511,fa7cdfad1a5aaf8370ebeda47a1ff1c3,3d44e8d90c2537e430a22aa71b868ba5,2022-08-26,0,NaN,0.0,56.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,9701a1c165dd9420816bfec5edd6c2b1,6d795bb9e527e367063c0044c4fc16a6,1,4,8,NaN,1
853937,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ceb2db5108750cf3c552ec2421b820c0,2022-08-06,1,NaN,0.0,82.0,82.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,2df45244f09369e16ea3f9117ca45157,aed4ae17a0ef8ac642419ef24e6709da,1,5,8,NaN,NaN
853938,fa7cdfad1a5aaf8370ebeda47a1ff1c3,ceb2db5108750cf3c552ec2421b820c0,2022-08-07,0,NaN,0.0,216.0,0.0,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1,c51ce410c124a10e0db5e4b97fc2af39,2df45244f09369e16ea3f9117ca45157,aed4ae17a0ef8ac642419ef24e6709da,1,6,8,NaN,NaN


In [48]:
new_df.isna().sum()

st_id                           0
pr_sku_id                       0
date                            0
pr_sales_type_id                0
pr_sales_in_units             652
pr_promo_sales_in_units         0
pr_sales_in_rub                 9
pr_promo_sales_in_rub           0
st_city_id                      0
st_division_code                0
st_type_format_id               0
st_type_loc_id                  0
st_type_size_id                 0
st_is_active                    0
pr_group_id                     0
pr_cat_id                       0
pr_subcat_id                    0
pr_uom_id                       0
day_of_week                     0
month                           0
one_unit_price                655
is_weekend                 173119
dtype: int64

In [49]:
new_df = new_df.dropna(subset=['pr_sales_in_units'])

In [50]:
new_df.to_csv (r'D:\yandex\new_data.csv', index= False )

In [51]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 877962 entries, 0 to 878613
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   st_id                    877962 non-null  object        
 1   pr_sku_id                877962 non-null  object        
 2   date                     877962 non-null  datetime64[ns]
 3   pr_sales_type_id         877962 non-null  int64         
 4   pr_sales_in_units        877962 non-null  float64       
 5   pr_promo_sales_in_units  877962 non-null  float64       
 6   pr_sales_in_rub          877959 non-null  float64       
 7   pr_promo_sales_in_rub    877962 non-null  float64       
 8   st_city_id               877962 non-null  object        
 9   st_division_code         877962 non-null  object        
 10  st_type_format_id        877962 non-null  int64         
 11  st_type_loc_id           877962 non-null  int64         
 12  st_type_size_id 

In [52]:
new_df = new_df.sort_values(by='date')

In [53]:
columns = ['pr_promo_sales_in_units','pr_sales_in_rub','pr_promo_sales_in_rub']

In [54]:
# тут удаляем столбцы, которые могут вызвать утечки

new_df2 = new_df.drop(columns, axis=1)

In [55]:
# Функция метрики WAPE
def wape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

In [56]:
# Создание scorer для использования в cross_val_score
wape_scorer = make_scorer(wape, greater_is_better=False)

In [57]:
cat_features = ['st_id', 'pr_sku_id', 'st_city_id', 'st_division_code', 'pr_group_id', 'pr_cat_id', 'pr_subcat_id']

In [71]:
# Функция кросс-валидации временных рядов
def time_series_cross_validation(data, target_column="pr_sales_in_units"):
    
    # Задаем размеры обучающего, валидационного и тестового наборов
    train_size = timedelta(days=6*30)
    validation_size = timedelta(weeks=2)
    test_size = timedelta(weeks=2)
    
    # Задаем шаг сдвига
    step_size=timedelta(weeks=2)
    
    # Задаем начальную дату
    start_date = pd.to_datetime("01.08.2022")

    # Параметры для RandomizedSearchCV
    param_distributions = {
        'iterations': [500, 1000, 1500],
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.001, 0.01, 0.1],
        'l2_leaf_reg': [1, 3, 5, 7, 9]
    }
    
    results = []

    while start_date + train_size + validation_size + test_size <= pd.to_datetime("18.07.2023"):
        
        train_end = start_date + train_size
        validation_end = train_end + validation_size
        test_end = validation_end + test_size

        train = data[(data['date'] >= start_date) & (data['date'] < train_end)]
        validation = data[(data['date'] >= train_end) & (data['date'] < validation_end)]
        test = data[(data['date'] >= validation_end) & (data['date'] < test_end)]

        # Проверка на наличие достаточного количества данных
        if len(train) == 0 or len(validation) == 0 or len(test) == 0:
            print(f"Пропускаем из за отутствия данных. Train: {start_date} - {train_end}, Validation: {train_end} - {validation_end}")
            start_date += validation_size
            continue

        X_train, y_train = train.drop(columns=target_column), train[target_column]
        X_validation, y_validation = validation.drop(columns=target_column), validation[target_column]
        X_test, y_test = test.drop(columns=target_column), test[target_column]

        

        model = CatBoostRegressor(cat_features=cat_features, verbose=1)

        # Подбор гиперпараметров
        searcher = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, 
                                      scoring='neg_mean_absolute_error', n_iter=3, verbose=1, random_state=42)
        searcher.fit(pd.concat([X_train, X_validation]), pd.concat([y_train, y_validation]))

        # Обучение с лучшими параметрами
        best_model = searcher.best_estimator_
        best_model.fit(pd.concat([X_train, X_validation]), pd.concat([y_train, y_validation]))

        # Предсказание на тестовом наборе
        predictions = best_model.predict(X_test)
        
        # Расчет WAPE
        error = wape(y_test, predictions)
        results.append(error)

        print(f"Wape на интервале с {start_date} по {test_end}: {error:.4f}")

        start_date += step_size

    return results



In [72]:
%%time
results = time_series_cross_validation(new_df)

Skipping iteration due to insufficient data. Train: 2022-01-08 00:00:00 - 2022-07-07 00:00:00, Validation: 2022-07-07 00:00:00 - 2022-07-21 00:00:00
Skipping iteration due to insufficient data. Train: 2022-01-22 00:00:00 - 2022-07-21 00:00:00, Validation: 2022-07-21 00:00:00 - 2022-08-04 00:00:00
Fitting 5 folds for each of 3 candidates, totalling 15 fits
0:	learn: 9.5070024	total: 31.5ms	remaining: 31.4s
1:	learn: 9.4486262	total: 63.6ms	remaining: 31.8s
2:	learn: 9.3891014	total: 95ms	remaining: 31.6s
3:	learn: 9.3304077	total: 126ms	remaining: 31.4s
4:	learn: 9.2702366	total: 158ms	remaining: 31.4s
5:	learn: 9.2123934	total: 188ms	remaining: 31.1s
6:	learn: 9.1543853	total: 217ms	remaining: 30.7s
7:	learn: 9.0981009	total: 252ms	remaining: 31.3s
8:	learn: 9.0421209	total: 287ms	remaining: 31.6s
9:	learn: 8.9860597	total: 319ms	remaining: 31.6s
10:	learn: 8.9313248	total: 346ms	remaining: 31.1s
11:	learn: 8.8765229	total: 378ms	remaining: 31.1s
12:	learn: 8.8209722	total: 408ms	remai

KeyboardInterrupt: 

In [70]:
print(results)

[0.05714589278736344, 0.04712656270712338]


In [61]:
print(np.mean(results))

0.05213622774724341


In [62]:
%%time
results2 = time_series_cross_validation(new_df2)

Skipping iteration due to insufficient data. Train: 2022-01-08 00:00:00 - 2022-07-07 00:00:00, Validation: 2022-07-07 00:00:00 - 2022-07-21 00:00:00
Skipping iteration due to insufficient data. Train: 2022-01-22 00:00:00 - 2022-07-21 00:00:00, Validation: 2022-07-21 00:00:00 - 2022-08-04 00:00:00
Fitting 5 folds for each of 5 candidates, totalling 25 fits
0:	learn: 9.5506418	total: 33.2ms	remaining: 33.2s
1:	learn: 9.5350244	total: 66.3ms	remaining: 33.1s
2:	learn: 9.5191787	total: 93.1ms	remaining: 30.9s
3:	learn: 9.5035731	total: 119ms	remaining: 29.6s
4:	learn: 9.4887847	total: 150ms	remaining: 29.8s
5:	learn: 9.4734325	total: 174ms	remaining: 28.8s
6:	learn: 9.4591329	total: 197ms	remaining: 27.9s
7:	learn: 9.4391748	total: 224ms	remaining: 27.7s
8:	learn: 9.4167150	total: 255ms	remaining: 28.1s
9:	learn: 9.3963233	total: 281ms	remaining: 27.9s
10:	learn: 9.3765969	total: 304ms	remaining: 27.4s
11:	learn: 9.3539424	total: 327ms	remaining: 26.9s
12:	learn: 9.3332677	total: 353ms	rem

In [69]:
print(results2)

[0.5849641073976624, 0.5841281102088732]


In [64]:
print(np.mean(results2))

0.5845461088032677
